## Taller 05: Herramientas para Indexación

Integrantes: Michael Pillaga, Alexis Vera, Jordy Quishpe

### Preprocesamiento del texto

In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import nltk

# Descargar los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Saitama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saitama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Saitama\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Normalizacion de texto

In [3]:
# Cargar el archivo CSV
file_path = './wiki_movie_plots_deduped.csv'
df = pd.read_csv(file_path)

# Inicializar el lematizador y stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función para normalizar el texto
def normalize_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación y caracteres especiales
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenizar
    tokens = word_tokenize(text)
    # Eliminar stopwords y lematizar las palabras
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

# Aplicar la normalización a las tramas de las películas
df['Normalized_Plot'] = df['Plot'].apply(normalize_text)

# Crear el índice invertido
inverted_index = defaultdict(set)

for idx, row in df.iterrows():
    title = row['Title']
    for word in row['Normalized_Plot']:
        inverted_index[word].add(title)



In [5]:
# Consultar el índice invertido para la palabra 'cyborg'
word = 'cyborg'  # Palabra que quieres buscar
movies_with_word = inverted_index[word] if word in inverted_index else set()

# Mostrar solo las primeras 5 películas
print(f"Primeras 5 películas que contienen la palabra '{word}':")
print(list(movies_with_word)[:5])


Primeras 5 películas que contienen la palabra 'cyborg':
['Future War', 'The Time Guardian', 'Kamen Rider the First', '964 Pinocchio', 'Godzilla: Tokyo S.O.S.']


###  PARTE 2: Usar Whoosh para Indexacion 

In [7]:
pip install whoosh


  Obtaining dependency information for whoosh from https://files.pythonhosted.org/packages/ba/19/24d0f1f454a2c1eb689ca28d2f178db81e5024f42d82729a4ff6771155cf/Whoosh-2.7.4-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/468.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/468.8 kB ? eta -:--:--
    --------------------------------------- 10.2/468.8 kB ? eta -:--:--
   -- ------------------------------------ 30.7/468.8 kB 435.7 kB/s eta 0:00:02
   ------------ --------------------------- 143.4/468.8 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------  460.8/468.8 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 468.8/468.8 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
import os
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Cargar el archivo CSV
file_path = './wiki_movie_plots_deduped.csv'
df = pd.read_csv(file_path)

# Inicializar el lematizador y stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función para normalizar el texto
def normalize_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación y caracteres especiales
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenizar
    tokens = word_tokenize(text)
    # Eliminar stopwords y lematizar las palabras
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

# Aplicar la normalización a las tramas de las películas
df['Normalized_Plot'] = df['Plot'].apply(normalize_text)

# Definir el esquema del índice con los campos 'Title' y 'Plot'
schema = Schema(
    title=TEXT(stored=True),  # Título de la película
    plot=TEXT(stored=True)    # Trama de la película
)

# Directorio donde se guardará el índice
index_dir = 'movie_index'

# Verificar si ya existe un índice en el directorio. Si no, crearlo
if not os.path.exists(index_dir):
    os.mkdir(index_dir)
    ix = create_in(index_dir, schema)
else:
    ix = open_dir(index_dir)

# Función para agregar películas al índice
def add_movies_to_index(movies):
    writer = ix.writer()  # Crear el escritor para agregar documentos
    
    for idx, row in movies.iterrows():
        title = row['Title']
        plot = row['Normalized_Plot']  # Usamos la trama normalizada
        
        # Convertir la lista de palabras de la trama de nuevo a una cadena
        plot_text = " ".join(plot)
        
        # Agregar cada película como documento en el índice
        writer.add_document(title=title, plot=plot_text)
    
    # Guardar los documentos en el índice
    writer.commit()

# Agregar todas las películas al índice
add_movies_to_index(df)


Películas relacionadas con 'cyborg':
Title: Cyborg 3: The Recycler
Plot: set desolate post apocalyptic world thriving golden age man cyborg ended cyborg hunted part cash haje female cyborg learns doc edford margaret avery somehow pregnant search fabled city cytown find eva...
----------------------------------------
Title: Cyborg Cop II
Plot: crime bos jesse starkraven lead gang attack drug den cooperating starkravens demand assault quickly turn hostage situation police arrive renegade dea agent jack ryan arrives attempt negotiate starkrav...
----------------------------------------
Title: Cyborg 009
Plot: nine people around world kidnapped evil black ghost organization undergo experiment turn cyborg superhuman power nine cyborg band together order stop black ghost achieving goal starting next world war...
----------------------------------------
Title: Cyborg 009: Monster War
Plot: nine people around world kidnapped evil black ghost organization undergo experiment turn cyborg superhum

### Consulta usando Woosh

In [2]:
# Realizar una consulta en el índice
query_string = "cyborg"  # Palabra clave para la consulta

with ix.searcher() as searcher:
    query = QueryParser("plot", ix.schema).parse(query_string)
    
    # Ejecutar la búsqueda
    results = searcher.search(query, limit=5)  # Limitamos a 5 resultados
    
    # Mostrar los resultados
    print(f"Películas relacionadas con '{query_string}':")
    for result in results:
        print(f"Title: {result['title']}")
        print(f"Plot: {result['plot'][:200]}...")  # Mostrar solo una parte de la trama
        print("-" * 40)


Películas relacionadas con 'cyborg':
Title: Cyborg 3: The Recycler
Plot: set desolate post apocalyptic world thriving golden age man cyborg ended cyborg hunted part cash haje female cyborg learns doc edford margaret avery somehow pregnant search fabled city cytown find eva...
----------------------------------------
Title: Cyborg Cop II
Plot: crime bos jesse starkraven lead gang attack drug den cooperating starkravens demand assault quickly turn hostage situation police arrive renegade dea agent jack ryan arrives attempt negotiate starkrav...
----------------------------------------
Title: Cyborg 009
Plot: nine people around world kidnapped evil black ghost organization undergo experiment turn cyborg superhuman power nine cyborg band together order stop black ghost achieving goal starting next world war...
----------------------------------------
Title: Cyborg 009: Monster War
Plot: nine people around world kidnapped evil black ghost organization undergo experiment turn cyborg superhum

### PARTE 3: Elasticsearch para Indexación y Recuperación

In [1]:
pip install elasticsearch

  Obtaining dependency information for elasticsearch from https://files.pythonhosted.org/packages/1e/82/832ff4bdb53429af0025f5032c8b4f3ba18915e08ce16fc55aa09e900e26/elasticsearch-8.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for elastic-transport<9,>=8.15.1 from https://files.pythonhosted.org/packages/2a/0d/2dd25c06078070973164b661e0d79868e434998391f9aed74d4070aab270/elastic_transport-8.17.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/571.2 kB ? eta -:--:--
   ---------- ----------------------------- 143.4/571.2 kB 4.2 MB/s eta 0:00:01
   ----------------------------- ---------- 419.8/571.2 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------  563.2/571.2 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------  563.2/571.2 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------  563.2/571.2 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 571.2/571.2 kB 2.4 MB/s eta 0:00:00
   ---

#### Conexion con Docker

Para la conexion con Docker usamos los siguientes comandos desde la cmd:

docker network create elastic

docker pull docker.elastic.co/elasticsearch/elasticsearch:8.17.0

docker run -d --name elasticsearch -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.17.0


In [5]:
from elasticsearch import Elasticsearch

# Conexión al cliente Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Verificar si está conectado
if es.ping():
    print("Conexión exitosa a Elasticsearch")
else:
    print("Error al conectar con Elasticsearch")

Conexión exitosa a Elasticsearch


In [6]:
# Crear un índice con campos 'Title' y 'Plot'
index_name = "movies"  # Nombre del índice

# Definir el esquema del índice
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},  # Campo de texto completo
            "plot": {"type": "text"}    # Campo de texto completo
        }
    }
}

# Crear el índice si no existe
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Índice '{index_name}' creado con éxito.")
else:
    print(f"El índice '{index_name}' ya existe.")


Índice 'movies' creado con éxito.


#### Paso 2: Indexar documentos (películas)
Supongamos que tienes un DataFrame df con las columnas Title y Plot.

In [8]:
# Indexar documentos en Elasticsearch
def index_documents(es, index_name, df):
    for idx, row in df.iterrows():
        # Crear un documento con los campos Title y Plot
        doc = {
            "title": row['Title'],
            "plot": row['Plot']
        }
        
        # Usar el título como identificador único
        es.index(index=index_name, id=idx+1, body=doc)
    
    # Mostrar mensaje una vez que todos los documentos se hayan indexado
    print(f"Todas las {len(df)} películas fueron indexadas correctamente.")

# Llamar a la función para indexar documentos
index_documents(es, index_name, df)


Todas las 34886 películas fueron indexadas correctamente.


### Realizar Consultas
Elasticsearch permite realizar búsquedas avanzadas utilizando el campo plot para buscar términos específicos como "time travel" o "genetic engineering". A continuación, te muestro cómo hacerlo.

In [9]:
# Consulta para buscar películas relacionadas con "time travel"
query = {
    "query": {
        "match": {
            "plot": "time travel"
        }
    }
}

# Ejecutar la consulta
response = es.search(index=index_name, body=query, size=5)  # Limitar a 5 resultados
print("Películas relacionadas con 'time travel':")
for hit in response['hits']['hits']:
    print(f"Título: {hit['_source']['title']}")
    print(f"Trama: {hit['_source']['plot']}")
    print("-" * 40)


C:\Users\Saitama\AppData\Local\Temp\ipykernel_14112\1831387768.py:11: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=query, size=5)  # Limitar a 5 resultados


Películas relacionadas con 'time travel':
Título: Time Chasers
Trama: Physics teacher and amateur pilot Nick Miller (Matthew Bruch) has finally completed his quest of enabling time travel, via a Commodore 64 and his small airplane. After being inspired by a television commercial for GenCorp, he uses a ruse to bring out both a GenCorp executive and a reporter from a local paper. To Nick's surprise, the reporter is Lisa Hansen (Bonnie Pritchard), an old high school flame. One trip to 2041 later and Gencorp's executive, Matthew Paul (Peter Harrington), quickly arranges Nick a meeting with CEO J.K. Robertson (George Woodard). Impressed by the potential of time travel, Robertson offers Nick a licensing agreement on the technology.
The following week, Nick and Lisa meet at the supermarket and go on a date to the 1950s. However, another trip to 2041 reveals that GenCorp abused Nick's time travel technology, creating a dystopian future. In an attempt to tell J.K. about how GenCorp inadvertentl